In [16]:
from dask.distributed import Client, progress, LocalCluster
import dask.array as da

from matplotlib import pyplot as plt

import dask.array.image
import numpy as np
import itk
import dask.array as da
import dask

import time

## Specify the distributed computing resources

Dask supports running on HPC MPI, HPC job schedulers, cloud-based clusters, or a set of systems connected over SSH.

As a first step, we specify how to connect to our distributed computing resources.

In this example, we will test with a client that just uses the multiple cores on our local systems.

In [2]:
## run dask-scheduler at a scheduler machine
## run dask-worker ip:8786 at a worker
# local_cluster = LocalCluster(n_workers=8, processes=False, memory_limit='4G')
# client = Client(local_cluster)
client = Client(n_workers=4, processes=False)
client

Client Scheduler: inproc://10.10.10.135/23312/1 Dashboard: http://10.10.10.135:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.50 GB


In [14]:
# An example image processing pipeline with a parameter
def my_processing_pipeline(image_chunk, radius=2):
    import itk
    
    denoised = itk.median_image_filter(image_chunk, radius=radius)
    return denoised

In [4]:
input_filepath = '../data/bead_pack.tif'

In [22]:
image = itk.imread(input_filepath)
image = np.asarray(image)

## Option 0: Non-distributed

In [23]:
start = time.time()

denoised = my_processing_pipeline(image, radius=2)

elapsed = time.time() - start
print(elapsed, 'seconds')

1.9957807064056396 seconds


## Option 1: Dask Client.submit

### Submit the processing pipeline and image as a task

In [24]:
start = time.time()

denoised = client.submit(my_processing_pipeline, np.asarray(image)).result()

elapsed = time.time() - start
print(elapsed, 'seconds')

2.1309280395507812 seconds


In [25]:
# Chunk the data into a Dask array.
chunked_data = da.from_array(image, chunks=(5, 200, 200))  ## 40 blocks
chunked_data

,Array,Chunk
Bytes,8.00 MB,200.00 kB
Shape,"(200, 200, 200)","(5, 200, 200)"
Count,41 Tasks,40 Chunks
Type,uint8,numpy.ndarray


### Scatter the data across the clients

In [26]:
# Scatter the data across the clients
scattered_data = client.scatter(chunked_data, broadcast=True)

start = time.time()

denoised = client.submit(my_processing_pipeline, scattered_data).result()

elapsed = time.time() - start
print(elapsed, 'seconds')

2.0224642753601074 seconds


## Option 2: Dask Array map_blocks

In [30]:
start = time.time()

denoised = da.map_blocks(my_processing_pipeline,
              chunked_data,
              radius=2,
              dtype=chunked_data.dtype)
denoised = denoised.compute()

elapsed = time.time() - start
print(elapsed, 'seconds')

2.6059038639068604 seconds


## Option 3: Dask Array map_overlap

Extend the processed region, then trim the result.

In [31]:
start = time.time()

denoised = da.map_overlap(my_processing_pipeline,
              chunked_data,
              depth=2,
              trim=True,
              radius=2,
              dtype=chunked_data.dtype)
denoised = denoised.compute()

elapsed = time.time() - start
print(elapsed, 'seconds')

6.3604896068573 seconds
